In [167]:
from Extractor import *
from glob import glob
import pandas as pd
import re
from tqdm import tqdm

In [168]:
seq_length = 40
class_limit = None
proc_csv = pd.read_csv('../Vis/processedAnnotations.csv')
path = '/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/'

In [169]:
proc_csv.drop(['video_url', 'filename', 'configuracao_mao'], axis=1, inplace=True)
proc_csv['path_10'] = proc_csv['palavra'].map(lambda x:f"{path}{str(x)[0]}/{str(x)}/DEResult_10")
proc_csv['path_5'] = proc_csv['palavra'].map(lambda x:f"{path}{str(x)[0]}/{str(x)}/DEResult_5")
proc_csv['path_15'] = proc_csv['palavra'].map(lambda x:f"{path}{str(x)[0]}/{str(x)}/DEResult_15")

In [170]:
def getFrameNumber(path):
    return int(re.search(r".*\/frame(\d+)\.jpg", path).group(1))

In [171]:
def getFiles(path):
    files = list(glob(path + "/*.jpg"))
    files.sort(key=getFrameNumber)
    return files

In [172]:
proc_csv['files_list_10'] = proc_csv['path_10'].map(getFiles)
proc_csv['files_list_5'] = proc_csv['path_5'].map(getFiles)
proc_csv['files_list_15'] = proc_csv['path_15'].map(getFiles)

In [178]:
proc_csv.drop(['path_10', 'path_15', 'path_5'], axis=1, inplace=True)

In [180]:
proc_csv.to_pickle('dataset_with_file_list.pkl')

In [152]:
model = Extractor()

In [153]:
def buildSequence(frameList):
    sequence = []
    for image in frameList:
        features = model.extract(image)
        sequence.append(features)
    return np.array(sequence)

In [154]:
proc_csv.set_index('palavra', inplace=True)
folder = '/home/fabiana/Desktop/projeto-final-src/Classifier/InceptionV3_Features/'
sample = proc_csv.head()

In [106]:
%%time
for palavra, frameList in tqdm(proc_csv['files_list'].iteritems(), total=len(proc_csv)):
    seq = buildSequence(frameList)
    np.save(folder + palavra, sequence)

A
ABACATE
ABACAXI
ABAFAR
ABAIXO
CPU times: user 30.8 s, sys: 23.4 s, total: 54.2 s
Wall time: 17.2 s
